In [2]:
from ioMicro import *

In [3]:
data_folder= r'S:\ZaneTrit_3_3_2023'

In [4]:
# map all the hybes
hybes =  glob.glob(data_folder+os.sep+'H*')
# map all the fovs
fovs = [os.path.basename(fl)for fl in glob.glob(hybes[0]+os.sep+'*.zarr')]
def get_Hi(fld): 
    try: return int(os.path.basename(fld)[1:]) 
    except: return -1
    
hybes = np.array(hybes)[np.argsort([get_Hi(hybe) for hybe in hybes ])]

In [5]:
## Quickly check an image
fov = fovs[20]
fl = hybes[0]+os.sep+fov
im = read_im(fl)
import napari
napari.view_image(im,contrast_limits=[0,50000])

C:\Users\Scope2\anaconda3\lib\site-packages\napari_tools_menu\__init__.py:179: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 999.5, 999.5), zoom=0.2432, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0, 0.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=4, ndisplay=2, last_used=0, range=((0.0, 4.0, 1.0), (0.0, 75.0, 1.0), (0.0, 2000.0, 1.0), (0.0, 2000.0, 1.0)), current_step=(2, 37, 1000, 1000), order=(0, 1, 2, 3), axis_labels=('0', '1', '2', '3')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'im' at 0x2390d8a7df0>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10, unit=None), text_overlay=TextOverlay(visible=False, color=(0.5, 0.5, 0.5, 1.0), font_size=10, position=<TextOverlayPosition.TOP_LEFT: 'top_left'>, text=''), overlays=Overlays(interaction_box=InteractionBox(points=None, show=False, show_handle=F

In [6]:
imGFP1 = np.array(im[0],dtype=np.float32) #GFP 750 half
imGFP2 = np.array(im[1],dtype=np.float32) #GFP cy5 half

In [7]:
#Subtract local background
imGFP1_ = norm_slice(imGFP1, s=30)
imGFP2_ = norm_slice(imGFP2, s=30)

In [8]:
#fit both GFPs
Xh1 = get_local_max(imGFP1_,750,im_raw=imGFP1,delta=1,delta_fit=3)
Xh2 = get_local_max(imGFP2_,750,im_raw=imGFP2,delta=1,delta_fit=3)
#zc-center positions in pixels for z,xc,yc,bk- minimum signal within radius,a-total sum of signal within sphere of radius 3 pixels ,habs - max brightness of local max in oridgianl image,hn - correlation with expected PSF,h- maxium brightness in the subtracted

In [11]:
h = Xh1[:,-1]
cor = Xh1[:,-2]
cor

array([0.23597258, 0.20424256, 0.18279964, ..., 0.39054113, 0.34681699,
       0.3341899 ])

In [15]:
viewer = napari.Viewer()

viewer.add_image(imGFP1_,contrast_limits=[0,5000],blending='additive',colormap='green')
viewer.add_image(imGFP2_,contrast_limits=[0,5000],blending='additive',colormap='magenta')
hcutoff = 3000
h = Xh1[:,-1]
cor = Xh1[:,-2]
keep = (h>hcutoff)&(cor>0.5)
h = Xh1[keep,-1]
X = Xh1[keep,:3]
size = 5#np.clip(h/np.percentile(h,99),0,1)*10
X1 = X
viewer.add_points(X,size=size,face_color=[0,0,0,0],edge_color='g')
h = Xh2[:,-1]
cor = Xh2[:,-2]
keep = (h>hcutoff)&(cor>0.5)
h = Xh2[keep,-1]
X = Xh2[keep,:3]

X2 = X
size = 5#np.clip(h/np.percentile(h,99),0,1)*10

viewer.add_points(X,size=size,face_color=[0,0,0,0],edge_color='m')


h = Xh1[:,-1]
cor = Xh1[:,-2]
keep = (h>hcutoff)&(cor<0.25)
Xbad = Xh1[keep,:3]

h = Xh2[:,-1]
cor = Xh2[:,-2]
keep = (h>hcutoff)&(cor<0.25)
Xbad = np.concatenate([Xbad,Xh2[keep,:3]],axis=0)
viewer.add_points(Xbad,size=size,face_color=[0,0,0,0],edge_color='r')

C:\Users\Scope2\anaconda3\lib\site-packages\napari_tools_menu\__init__.py:179: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Points layer 'Xbad' at 0x23920a48fd0>

In [ ]:
X

In [69]:
X1.shape,X2.shape

((2158, 3), (1441, 3))

In [16]:
from scipy.spatial import KDTree
tree1 = KDTree(X1)
dist,ind1 = tree1.query(X2)
ind2 = np.arange(len(X2))
keep = dist<4
ind1,ind2=ind1[keep],ind2[keep]
X1_,X2_ = X1[ind1],X2[ind2]

In [18]:

tree_bad = KDTree(Xbad)
d1,ind1=tree_bad.query(X1_)
#d2,ind2=tree_bad.query(X2_)
X1f = X1_[d1>20]
X2f = X2_[d1>20]


In [21]:
v = napari.view_points(X1f,face_color='g',blending='additive')
v.add_points(X2f,face_color='r',blending='additive')

C:\Users\Scope2\anaconda3\lib\site-packages\napari_tools_menu\__init__.py:179: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Points layer 'X2f' at 0x23922966e50>

In [42]:
fl1 = hybes[0]+os.sep+fov
im1 = read_im(fl1)
im1dapi = np.array(im1[-1][(sz-ssz)//2:(sz+ssz)//2],dtype=np.float32)
dic_drift = {}
for hybe in hybes:
    fl2 = hybe+os.sep+fov
    im2 = read_im(fl2)
    ncols,sz,sx,sy = im1.shape
    ssz = 20

    im2dapi = np.array(im2[-1][(sz-ssz)//2:(sz+ssz)//2],dtype=np.float32)
    txyz,txyzs = get_txyz(im1dapi,im2dapi,sz_norm=30,sz = 500,nelems=5)
    dic_drift[fl2]=txyz
    print(txyz,txyzs)

[0 0 0] [array([0, 0, 0], dtype=int64), array([0, 0, 0], dtype=int64), array([0, 0, 0], dtype=int64), array([0, 0, 0], dtype=int64), array([0, 0, 0], dtype=int64)]
[  0  92 184] [array([  0,  93, 184], dtype=int64), array([ -1,  92, 183], dtype=int64), array([  0,  92, 184], dtype=int64), array([  0,  92, 184], dtype=int64), array([ -1,  93, 185], dtype=int64)]
[  0  16 138] [array([  0,  15, 138], dtype=int64), array([  0,  16, 138], dtype=int64), array([  0,  16, 138], dtype=int64), array([  0,  16, 138], dtype=int64), array([  0,  16, 138], dtype=int64)]
[  0 -15 110] [array([  0, -15, 110], dtype=int64), array([  0, -15, 110], dtype=int64), array([  0, -14, 110], dtype=int64), array([  0, -15, 110], dtype=int64), array([  0, -15, 110], dtype=int64)]
[  0   0 113] [array([  0,   0, 113], dtype=int64), array([  0,   0, 114], dtype=int64), array([  0,   1, 113], dtype=int64), array([  0,   0, 114], dtype=int64), array([  0,   0, 113], dtype=int64)]
[  0  -1 107] [array([  0,  -2, 107]

In [ ]:
pickle.dump(dic_drift)

In [33]:
for hybe in hybes:
    fl2 = hybe+os.sep+fov
    im2 = read_im(fl2)

In [40]:
import dask.array as da
im2T = da.roll(im2,-txyz,axis=[1,2,3])
viewer = napari.Viewer()
viewer.add_image(im1,contrast_limits=[0,20000])
viewer.add_image(im2T,contrast_limits=[0,20000])

C:\Users\Scope2\anaconda3\lib\site-packages\napari_tools_menu\__init__.py:179: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Image layer 'im2T' at 0x23948f23dc0>

In [39]:
txyz

array([  0,   0, 103])

In [34]:
im2.shape

(3, 75, 2000, 2000)

In [44]:
np.min(size)

0.0013134090048539237

In [36]:
Xh1.shape

(139292, 8)

In [31]:
#imGFP2
??norm_slice

In [7]:
#"Load" a fov


'Conv_zscan__20.zarr'

In [24]:
??read_im